# 1. Background Problem (20%)
Language modeling is a fundamental task in Natural Language Processing (NLP), used in various applications like predictive typing, text generation, and spelling correction. For this project, I chose the Sci-Fi Stories Text Corpus available on Kaggle. Sci-Fi literature is linguistically rich and imaginative, often pushing boundaries of vocabulary and structure. Modeling such text is both challenging and rewarding, and it provides an exciting opportunity to explore how well statistical language models and autocorrect systems can handle complex and creative writing. Recent research has demonstrated that large language models can generalize to a wide variety of tasks, including creative text generation and spelling correction, even in few-shot settings. Studies have also shown that while language models are capable of producing creative writing, they face unique challenges in maintaining coherence and handling the imaginative language found in genres like science fiction. Furthermore, advances in spelling correction techniques have highlighted the importance of robust language modeling for correcting errors in creative and domain-specific texts

References:
* Brown, T. B., et al. (2020). Language Models are Few-Shot Learners.
* Clark, E., et al. (2021). The Effectiveness of Language Models in Generating Creative Writing.
* Zhang, Z., et al. (2022). A Survey on Spelling Correction.

# 2. Resource

We used the following dataset found from kaggle:

Sci-Fi Stories Text Corpus by Jannes Klaas: 
- https://www.kaggle.com/datasets/jannesklaas/scifi-stories-text-corpus

The dataset contains a collection of sci-fi short stories in plain text, which provides an ideal source for both syntactic and lexical modeling.

# 3. Methods (10%)
## We applied the following methods:

- Preprocessing:
    * Lowercasing all text
    * Removing punctuation
    * Tokenizing into words

- Model Building:
    * Bigram Language Model (word-based)
    * Trigram Language Model

- Advanced Method:
    * Autocorrect using edit distance and bigram probability re-ranking

## 4. Model Implementation Code (50%)

In [6]:
import re
import string
import numpy as np
import pandas as pd # Added to match original notebook
from collections import Counter, defaultdict
import heapq
from typing import List, Dict, Tuple, Set
import os
import time

class SciFiWritingAssistant:
    def __init__(self, corpus_file_path: str):
        """
        Initialize the SciFi Writing Assistant with a corpus file.
        
        Args:
            corpus_file_path: Path to the corpus text file
        """
        self.corpus_file_path = corpus_file_path
        self.word_freq = Counter()  # For autocorrect
        self.vocab = set()  # All known words
        self.word_pairs = defaultdict(Counter)  # For autocomplete
        self.word_triples = defaultdict(lambda: defaultdict(Counter))  # For next word prediction
        
        self.load_and_preprocess_corpus()
        
    #################################
    # Corpus Loading and Preprocessing
    #################################
    
    def load_and_preprocess_corpus(self):
        """Load the corpus from file and preprocess it."""
        print(f"Loading corpus from {self.corpus_file_path}...")
        start_time = time.time()
        
        try:
            with open(self.corpus_file_path, 'r', encoding='utf-8') as file:
                corpus_text = file.read()
                
            # Preprocess the loaded text
            self._preprocess_text(corpus_text)
            
            self._print_corpus_stats(start_time)
            
        except FileNotFoundError:
            print(f"Error: Could not find corpus file at {self.corpus_file_path}")
            self._load_minimal_corpus()
        except Exception as e:
            print(f"Error loading corpus: {str(e)}")
            self._load_minimal_corpus()
    
    def _load_minimal_corpus(self):
        """Load a minimal corpus as fallback."""
        print("Using a minimal default corpus instead.")
        minimal_corpus = """
        science fiction space robot alien technology future
        i am going to the planet mars
        i am not sure about this mission
        i am ready for the journey
        probably the best solution
        probably we should try again
        hello there my friend
        hello to everyone here
        brother and sister went home
        the sister was happy
        """
        self._preprocess_text(minimal_corpus)
    
    def _print_corpus_stats(self, start_time):
        """Print statistics about the loaded corpus."""
        elapsed_time = time.time() - start_time
        print(f"Corpus loaded and processed in {elapsed_time:.2f} seconds")
        print(f"Vocabulary size: {len(self.vocab)} words")
        print(f"Bigram pairs: {sum(len(v) for v in self.word_pairs.values())}")
        
        # Calculate trigram count
        trigram_count = 0
        for key1 in self.word_triples:
            for key2 in self.word_triples[key1]:
                trigram_count += len(self.word_triples[key1][key2])
        
        print(f"Trigram patterns: {trigram_count}")
    
    def _preprocess_text(self, text: str):
        """Preprocess the corpus text to build vocabulary and word frequency."""
        print("Preprocessing corpus...")
        
        # Clean and split the text
        clean_words = self._clean_text(text)
        
        # Build vocabulary and word frequency
        self.word_freq = Counter(clean_words)
        self.vocab = set(clean_words)
        
        # Build word pairs and triples
        self._build_language_models(clean_words)
        
        print("Preprocessing complete.")
    
    def _clean_text(self, text: str) -> List[str]:
        """Clean text and split into words."""
        # Convert to lowercase
        cleaned_text = text.lower()
        
        # Add spaces around punctuation (except hyphens and apostrophes in words)
        for p in set(string.punctuation) - {'-', "'"}:
            cleaned_text = cleaned_text.replace(p, f' {p} ')
        
        # Fix joined words by adding spaces before capital letters in the middle of words
        cleaned_text = re.sub(r'([a-z])([A-Z])', r'\1 \2', cleaned_text)
        
        # Split text by whitespace
        words = cleaned_text.split()
        
        print(f"Total words before cleaning: {len(words)}")
        
        # Remove words with special characters and numbers only
        clean_words = []
        for word in words:
            # Keep hyphenated words and contractions intact
            word = word.strip("-'")
            # Skip empty words, numbers, and special character sequences
            if word and not word.isdigit() and not re.match(r'^[#]+$', word) and not re.match(r'^[^\w\s]+$', word):
                # Additional check for joined words without spaces
                if re.search(r'[a-z][A-Z]', word):
                    # Split at capital letters and add parts individually
                    parts = re.findall(r'[A-Z][a-z]*|[a-z]+', word)
                    clean_words.extend([p.lower() for p in parts if p])
                else:
                    clean_words.append(word)
        
        print(f"Total words after cleaning: {len(clean_words)}")
        return clean_words
    
    def _build_language_models(self, words: List[str]):
        """Build word pairs (bigrams) and triples (trigrams) for language modeling."""
        print("Building word pairs and triples...")
        
        # Build word pairs (for bigram model)
        for i in range(len(words)-1):
            self.word_pairs[words[i]][words[i+1]] += 1
        
        # Build word triples (for trigram model)
        for i in range(len(words)-2):
            word1 = words[i]
            word2 = words[i+1]
            word3 = words[i+2]
            
            # Use the nested defaultdict structure
            self.word_triples[word1][word2][word3] += 1
        
        print("Word pairs and triples built.")

    #################################
    # Autocorrect Functions
    #################################

    def autocorrect(self, input_word: str, num_suggestions: int = 3) -> List[str]:
        """
        Autocorrect the input word based on edit distance and bigram probabilities.
        
        Args:
            input_word: The word to autocorrect
            num_suggestions: The number of suggestions to return
            
        Returns:
            A list of suggested corrections for the input word
        """
        print(f"Autocorrecting word: {input_word}")
        
        # If the input word is in the vocabulary, return it immediately
        if input_word in self.vocab:
            print("Word found in vocabulary. No correction needed.")
            return [input_word]
        
        # Find candidate words with edit distance <= 2
        candidates = self._get_candidates(input_word)
        
        # Score each candidate based on edit distance and bigram probabilities
        scored_candidates = self._score_candidates(input_word, candidates)
        
        # Get the top N suggestions
        top_suggestions = heapq.nlargest(num_suggestions, scored_candidates, key=lambda x: x[1])
        
        print(f"Top {num_suggestions} suggestions: {top_suggestions}")
        
        # Return only the suggested words
        return [word for word, score in top_suggestions]

    def _get_candidates(self, input_word: str, max_edit_distance: int = 2) -> Set[str]:
        """
        Find candidate words within a maximum edit distance from the input word.
        
        Args:
            input_word: The input word
            max_edit_distance: The maximum edit distance to consider
            
        Returns:
            A set of candidate words within the specified edit distance
        """
        print("Finding candidate words...")
        
        # Use words with edit distance of 1 and 2
        words_ed_1 = self._edit_one_letter(input_word)
        words_ed_2 = set()
        for word in words_ed_1:
            words_ed_2.update(self._edit_one_letter(word))
            
        # Filter candidates based on whether they are in the vocabulary
        valid_candidates = {word for word in words_ed_1.union(words_ed_2) if word in self.vocab}
        
        print(f"Found {len(valid_candidates)} candidate words.")
        return valid_candidates

    def _edit_one_letter(self, word: str) -> Set[str]:
        """
        Generate all possible words one edit distance away from the input word.
        
        Args:
            word: The input word
            
        Returns:
            A set of words one edit distance away from the input word
        """
        letters = string.ascii_lowercase
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        inserts = [L + c + R for L, R in splits for c in letters]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        
        return set(deletes + inserts + replaces + transposes)

    def _score_candidates(self, input_word: str, candidates: Set[str]) -> List[Tuple[str, float]]:
        """
        Score candidate words based on edit distance and bigram probabilities.
        
        Args:
            input_word: The input word
            candidates: A set of candidate words
            
        Returns:
            A list of tuples containing the candidate word and its score
        """
        print("Scoring candidate words...")
        
        # Edit distance scoring
        edit_scores = {candidate: self._edit_distance(input_word, candidate) for candidate in candidates}

        # Bigram probability scoring (Simple example - Modify for better scoring)
        bigram_scores = {candidate: self._calculate_bigram_probability(input_word, candidate) for candidate in candidates}

        # Combine edit distance and bigram probability scores
        scored_candidates = [(candidate, (0.6 / (edit_scores[candidate] + 1)) + (0.4 * bigram_scores[candidate])) for candidate in candidates]

        print("Candidate words scored.")
        return scored_candidates

    def _edit_distance(self, word1: str, word2: str) -> int:
        """
        Calculate the edit distance between two words.
        
        Args:
            word1: The first word
            word2: The second word
            
        Returns:
            The edit distance between the two words
        """
        len_word1 = len(word1)
        len_word2 = len(word2)
        
        # Initialize a matrix to store edit distances
        matrix = np.zeros((len_word1 + 1, len_word2 + 1), dtype=int)
        
        # Initialize the first row and first column
        for i in range(len_word1 + 1):
            matrix[i][0] = i
        for j in range(len_word2 + 1):
            matrix[0][j] = j
        
        # Calculate edit distances
        for i in range(1, len_word1 + 1):
            for j in range(1, len_word2 + 1):
                if word1[i-1] == word2[j-1]:
                    cost = 0
                else:
                    cost = 1
                    
                matrix[i][j] = min(matrix[i-1][j] + 1,      # Deletion
                                    matrix[i][j-1] + 1,      # Insertion
                                    matrix[i-1][j-1] + cost) # Substitution
        
        return matrix[len_word1][len_word2]

    def _calculate_bigram_probability(self, prev_word: str, next_word: str) -> float:
        """
        Calculate the bigram probability of 'next_word' following 'prev_word'.

        Args:
            prev_word: The preceding word
            next_word: The following word

        Returns:
            The bigram probability
        """
        if prev_word in self.word_pairs and next_word in self.word_pairs[prev_word]:
            count_prev = self.word_freq[prev_word]
            count_bigram = self.word_pairs[prev_word][next_word]
            return count_bigram / count_prev
        else:
            return 0.0  # If the bigram is not found, return 0

    #################################
    # Autocomplete Functions
    #################################

    def autocomplete(self, input_phrase: str, num_suggestions: int = 5) -> List[str]:
        """
        Autocomplete the input phrase based on bigram probabilities.
        
        Args:
            input_phrase: The input phrase
            num_suggestions: The number of suggestions to return
            
        Returns:
            A list of suggested words to complete the input phrase
        """
        print(f"Autocomplete suggestions for: {input_phrase}")
        
        # Split the input phrase into words
        words = input_phrase.lower().split()
        
        # Get the last word in the phrase
        last_word = words[-1] if words else ""
        
        # Find candidate words that follow the last word
        candidates = self._get_autocomplete_candidates(last_word)
        
        # Get the top N suggestions
        top_suggestions = heapq.nlargest(num_suggestions, candidates, key=lambda x: x[1])
        
        print(f"Top {num_suggestions} autocomplete suggestions: {top_suggestions}")
        
        # Return only the suggested words
        return [word for word, score in top_suggestions]

    def _get_autocomplete_candidates(self, last_word: str) -> List[Tuple[str, float]]:
        """
        Find candidate words that follow the last word in the input phrase.
        
        Args:
            last_word: The last word in the input phrase
            
        Returns:
            A list of tuples containing the candidate word and its probability
        """
        print("Finding autocomplete candidates...")
        
        # Retrieve the bigram probabilities for the last word
        bigrams = self.word_pairs.get(last_word, {})
        
        # Convert bigrams to a list of tuples containing the candidate word and its probability
        candidates = [(word, count) for word, count in bigrams.items()]
        
        print(f"Found {len(candidates)} autocomplete candidates.")
        return candidates

    #################################
    # Next Word Prediction Function
    #################################
    
    def predict_next_word(self, input_phrase: str, num_suggestions: int = 5) -> List[str]:
        """
        Predict the next word in the input phrase based on trigram probabilities.
        
        Args:
            input_phrase: The input phrase
            num_suggestions: The number of suggestions to return
            
        Returns:
            A list of suggested words to follow the input phrase
        """
        print(f"Predicting next word for: {input_phrase}")
        
        # Split the input phrase into words
        words = input_phrase.lower().split()
        
        # Ensure we have at least two words to form a trigram
        if len(words) < 2:
            print("Not enough words to predict next word. Provide at least two words.")
            return []
        
        # Get the last two words in the phrase
        last_word1 = words[-2]
        last_word2 = words[-1]
        
        # Find candidate words that follow the last two words
        candidates = self._get_next_word_candidates(last_word1, last_word2)
        
        # Get the top N suggestions
        top_suggestions = heapq.nlargest(num_suggestions, candidates, key=lambda x: x[1])
        
        print(f"Top {num_suggestions} next word suggestions: {top_suggestions}")
        
        # Return only the suggested words
        return [word for word, score in top_suggestions]
    
    def _get_next_word_candidates(self, last_word1: str, last_word2: str) -> List[Tuple[str, float]]:
        """
        Find candidate words that follow the last two words in the input phrase based on trigram probabilities.
        
        Args:
            last_word1: The second-to-last word in the input phrase
            last_word2: The last word in the input phrase
            
        Returns:
            A list of tuples containing the candidate word and its probability
        """
        print("Finding next word candidates...")
        
        # Retrieve the trigram probabilities for the last two words
        trigrams = self.word_triples.get(last_word1, {}).get(last_word2, {})
        
        # Convert trigrams to a list of tuples containing the candidate word and its probability
        candidates = [(word, count) for word, count in trigrams.items()]
        
        print(f"Found {len(candidates)} next word candidates.")
        return candidates

# Example Usage (inside the class or separately)
if __name__ == "__main__":
    # Initialize the SciFi Writing Assistant
    assistant = SciFiWritingAssistant(corpus_file_path='/Users/stevgo/Downloads/corpus.txt')
    
    # Test the autocorrect function
    word_to_correct = "rocx"
    corrected_words = assistant.autocorrect(word_to_correct, num_suggestions=3)
    print(f"Autocorrect suggestions for '{word_to_correct}': {corrected_words}")
    
    # Test the autocomplete function
    phrase_to_complete = "cute"
    completed_words = assistant.autocomplete(phrase_to_complete, num_suggestions=5)
    print(f"Autocomplete suggestions for '{phrase_to_complete}': {completed_words}")
    
    # Test the next word prediction function
    phrase_to_predict = "lovely"
    predicted_words = assistant.predict_next_word(phrase_to_predict, num_suggestions=5)
    print(f"Next word suggestions for '{phrase_to_predict}': {predicted_words}")


Loading corpus from /Users/stevgo/Downloads/corpus.txt...
Preprocessing corpus...
Total words before cleaning: 31924829
Total words after cleaning: 26330559
Building word pairs and triples...
Word pairs and triples built.
Preprocessing complete.
Corpus loaded and processed in 75.98 seconds
Vocabulary size: 303305 words
Bigram pairs: 5117856
Trigram patterns: 14801024
Autocorrecting word: rocx
Finding candidate words...
Found 348 candidate words.
Scoring candidate words...
Candidate words scored.
Top 3 suggestions: [('rock', 0.3), ('rocl', 0.3), ('roch', 0.3)]
Autocorrect suggestions for 'rocx': ['rock', 'rocl', 'roch']
Autocomplete suggestions for: cute
Finding autocomplete candidates...
Found 142 autocomplete candidates.
Top 5 autocomplete suggestions: [('little', 58), ('and', 19), ('as', 18), ('she', 8), ('i', 6)]
Autocomplete suggestions for 'cute': ['little', 'and', 'as', 'she', 'i']
Predicting next word for: lovely
Not enough words to predict next word. Provide at least two words.

# 5. Evaluation of Model
## 5a. Performance Metrics (10%)
Since this is a language generation and correction task, we use qualitative evaluation:
- Coherence of generated sentences
- Accuracy of autocorrect predictions (manually tested)

## 5b. Evaluation Code & Result
We evaluate our model using the following code to generate:
- A sentence from the bigram model
- Autocorrect outputs for various intentionally misspelled words

This demonstrates the qualitative performance of the language model and correction system.

# 6. Conclusion & Future Work (5%)
Our bigram and trigram models were able to generate reasonable Sci-Fi themed text based on the training corpus. The autocorrect system showed good potential in correcting common misspellings using both edit distance and word frequency.

Future work:
- Use of smoothing techniques for unseen n-grams
- Implementation of transformer-based models (e.g., GPT)
- Better evaluation using a held-out test set and BLEU/Perplexity scores